In [1]:
import pandas as pd
import numpy as np
import csv
from sqlalchemy import create_engine

In [2]:
user='root'
host='localhost'
port = 3306
database = '6Steps'
engine = create_engine(
        url="mysql+pymysql://{0}@{1}:{2}/{3}".format(
            user, host, port, database
        )
    )
conn = engine.connect()

# PDR

In [2]:
process_tech_df = pd.read_excel("../data/database table.xlsx", sheet_name="Process Tech table", usecols=["process_name","Process_tech_name"])
process_tech_df["process_name"].fillna(method="ffill", inplace=True)
process_tech_df.rename(columns={'Process_tech_name': 'process_tech_name'}, inplace=True)
process_tech_df.head()

,process_name,process_tech_name
0,Die_Attach,Soft solder
1,Die_Attach,Solder paste
2,Die_Attach,Diffusion solder
3,Die_Attach,Epoxy
4,Die_Attach,DAF


In [3]:
PBC_df = pd.read_excel("../data/database table.xlsx", 
                       sheet_name="PBC table", 
                       usecols=["PBC_name"], 
                       na_filter=False
                      )
PBC_df["PBC_color"] = PBC_df.PBC_name.apply(lambda row: row.split("(", 1)[-1].rstrip(")'") if not pd.isnull(row) else row)
PBC_df.head()

,PBC_name,PBC_color
0,NA,NA
1,standard_(green),green
2,limited_(yellow),yellow
3,restricted_(red),red
4,unclassified_(grey),grey


In [4]:
process_table = process_tech_df.join(PBC_df, how="cross")

In [5]:
process_table.head(3)

,process_name,process_tech_name,PBC_name,PBC_color
0,Die_Attach,Soft solder,NA,NA
1,Die_Attach,Soft solder,standard_(green),green
2,Die_Attach,Soft solder,limited_(yellow),yellow


In [6]:
process_table[(process_table['process_name']=="Die_Attach") & (process_table['process_tech_name']=="Diffusion solder") & (process_table['PBC_name']=="standard_(green)")]

,process_name,process_tech_name,PBC_name,PBC_color
11,Die_Attach,Diffusion solder,standard_(green),green


In [7]:
def table_to_sql (table, table_name, primary_key=None, float_cols=[], m=4, d=2, int_cols=[]):
    cols_size = []
    str_cols = [c for c in table.columns if c not in [primary_key]+float_cols+int_cols]
    for col in str_cols:
        max_len = int(max(table[col].astype(str).str.len()) * 1.5)
        cols_size.append((col, max_len))

    df_sql = """
    CREATE TABLE {table_name} 
    (
    {pk_col}{cols}{float_cols}{int_cols}{pk}
    );
    """.format(table_name=table_name,
               pk_col='{} INT NOT NULL AUTO_INCREMENT, '.format(primary_key) if primary_key else '',
               pk=', PRIMARY KEY ({})'.format(primary_key) if primary_key else '',
               cols=", ".join(["`{c}` varchar({s}) DEFAULT NULL".format(c=c, s=s) for c, s in cols_size]),
               float_cols=", "+", ".join(["`{c}` float({m},{d}) DEFAULT NULL".format(c=c,m=m,d=d) for c in float_cols]) if float_cols else "",
               int_cols=", "+", ".join(["`{c}` INT DEFAULT NULL".format(c=c) for c in int_cols]) if int_cols else ""
              )
    return(df_sql)

In [8]:
# conn.execute("drop table Process_PBHB_mapping;")
# conn.execute("drop table PDR_sessions;")

In [9]:
process_table.head(3)

,process_name,process_tech_name,PBC_name,PBC_color
0,Die_Attach,Soft solder,NA,NA
1,Die_Attach,Soft solder,standard_(green),green
2,Die_Attach,Soft solder,limited_(yellow),yellow


In [16]:
conn.execute("DROP TABLE Process_PBHB_mapping;")
conn.execute("DROP TABLE Process;")

conn.execute(table_to_sql(process_table, 'Process', 'process_id'))

process_table.to_sql('Process', conn, if_exists='append', index=False)

In [10]:
PBHB_wirebond = pd.read_excel("../data/Z8F67581994_PBHB wire bond.xlsx", 
                              sheet_name="PB IN_Process", 
                              header=1,
                              usecols=range(7,7+11),
                              skiprows=[2,3]
                            )
PBHB_wirebond.columns = ['PB_code'] + [' '.join(c.split()).lower() for c in PBHB_wirebond.columns[1:]]

PBHB_wirebond

,PB_code,requirements,parameter (input to pdr: output response),in adego rev. 7.0,verification method (input to pdr),development target (spec#) (input to pdr),sample size (input to pdr),take sample after process step,lab instruction (input in pdr),(primary) interaction to pb,motivation
0,INP01,Ball/bump bond dimension within bond pad opening,Ball/bump bond diameter\ni) Along X-axis\nii) ...,No,Light Optical Inspection,As per related specification requirement.\n\nT...,30 wires x number of lots @ each required proc...,After Interconnect,Work Instruction,Interconnect,"To avoid ball/bump bond out of bond pad, short..."
1,INP02,Wedge bond width within bond pad opening and l...,Wedge bond width\ni) At bond pad\nii) At lead ...,No,Light Optical Inspection,As per related specification requirement.\n(on...,30 wires x number of lots @ each required proc...,After Interconnect,Work Instruction,Interconnect,To avoid wedge bond out of bond pad/lead finge...
2,INP03,Stitch bond dimension within lead finger,Stitch bond dimension\ni) Stitch width\nii) St...,No,Light Optical Inspection,As per related specification requirement or MI...,30 wires x number of lots @ each required proc...,After Interconnect,Work Instruction,Interconnect,"To avoid broken stitch/wedge, lifted stitch/we..."
3,INP04,Verticle ball/bump bond dimension from bond pa...,Ball/bump bond height \n(Z-axis),Yes,Light Optical Inspection,As per related process specification requireme...,30 wires x number of lots @ each required proc...,After Interconnect,Work Instruction,Interconnect,"To avoid lifted ball/bump bond, smashed ball/b..."
4,INP05,Symmetrical/regular ball bond shape,Ball bond shape,Yes,Light Optical Inspection,Reject golf club formation if the center point...,30 wires x number of lots (shall include 4 di...,After Interconnect,Work Instruction,Interconnect,"To avoid lifted ball bond, smashed ball, dama..."
5,INP06,"Ball bond neck condition without crack, thinni...",Ball bond neck condition,Yes,SEM\n(Magnification: 1000x min.),No wire crack above wire neck (heat affected z...,Minimum 5 balls per unit from minimum 2 units ...,After Interconnect,Lab Instruction,Interconnect,"To avoid neck stress, wire thinning, necking, ..."
6,INP07,Ball/bump bond adhesion strength in terms of s...,Ball/bump bond shear force/strength,Yes,Ball Shear Test,As per related process specification or indust...,30 wires from minimum 5 units x number of lots...,After Interconnect,Work Instruction,Interconnect / Adhesion Promoter,Good ball/bump bond adhesion with bond pad.\n
7,INP08,Wedge bond adhesion strength in terms of shear...,Wedge shear force\ni) At bond pad\nii) At lead...,Yes,Wedge Shear Test,As per related process specification and Gener...,30 wires x number of lots @ each required proc...,After Interconnect,Work Instruction,Interconnect / Adhesion Promoter,Good wedge bond adhesion with bond pad.
8,INP09,Ball bond adhesion strength in terms of pull f...,Ball pull force,Yes,Ball Pull Test,As per related process specification and Gener...,30 wires from minimum 5 units x number of lots...,After Interconnect,Work Instruction,Interconnect / Adhesion Promoter,Good ball bond adhesion with bond pad\nTo avoi...
9,INP10,Stitch bond adhesion strength in terms of pull...,Stitch pull force,Yes,Stitch Pull Test,As per related process specification and Gener...,30 wires from minimum 5 units x number of lots...,After Interconnect,Work Instruction,Interconnect / Adhesion Promoter,Good stitch bond adhesion with lead finger\n\n...


In [11]:
PBHB_die_attach = pd.read_excel("../data/Z8F65720674_Die Attach.xlsx", 
                              sheet_name="PB DA_Process", 
                              header=1,
                              usecols=range(10,10+11),
                              skiprows=[2,3]
                            )
PBHB_die_attach.columns = list(PBHB_wirebond.columns)
PBHB_die_attach["PB_code"] = PBHB_die_attach["PB_code"].apply(lambda row: row.replace("\n", ""))
PBHB_die_attach

,PB_code,requirements,parameter (input to pdr: output response),in adego rev. 7.0,verification method (input to pdr),development target (spec#) (input to pdr),sample size (input to pdr),take sample after process step,lab instruction (input in pdr),(primary) interaction to pb,motivation
0,DAP01 /PUP01,Visual defects after die Pick-Up (die surface),"Traces of any visual damages, contamination an...",Yes,Light Optical Inspection,a. No cracked die\nb. No die-corner chipping\n...,30 dies \nADeGo:\n45 dies,After die Pick-Up,Work Instruction,Interconnect,Physical damages of a die could lead to variou...
1,DAP02,Visual defects after DA,"Traces of any visual damages, contamination an...",Yes,Light Optical Inspection,"a. No improper die placement, improper die pic...",10 dies per LF x 3 LFs\nADeGo:\n15 dies per LF...,After Die-Attach,Work Instruction,Interconnect,"any contamination, improper die/clip placement..."
2,DAP03,Translational and rotational offsets of die\n,1. x Offset\n2. y Offset\n3. Rotational offset,Yes,Light Optical Inspection,Min. Criteria:\n1. x offset <100 μm\n2. y offs...,10 dies per LF x 3 LFs \nADeGo:\n15 dies per L...,After Die-Attach,Work Instruction,Interconnect,Inaccurate die placement could lead to:\ni. In...
3,DAP04a,Coverage,Coverage,Yes,Light Optical Inspection,"Solder wire, solder paste, glue:\ncoverage > 7...",10 dies per LF x 3 LFs \nADeGo:\n15 dies per L...,After Die-Attach,Work Instruction,Interconnect,Insufficient coverage could lead to:\ni. Weak ...
4,DAP04b,"Coverage (solder wetting), solder bridging and...",1. Coverage \n2. Solder bridging\n3. Solder ex...,No,Light Optical Inspection,1. no insufficient solder coverage\n2. no sold...,30 dies on one LF,After Die-Attach,Work Instruction,Nil,Insufficient coverage could lead to:\ni. Weak ...
5,DAP04b,"Coverage (solder wetting), solder bridging and...",1. Mal-formation of solder lump / uneven squee...,No,X-ray,No mal-formation of solder lump / uneven squee...,5 units per map from 1 strip per lot,After Molding,Work Instruction,Nil,Insufficient coverage could lead to:\ni. Weak ...
6,DAP04c,Coverage,Coverage,No,X-RAY,as per respective failure catalog,10 dies per LF x 3 LFs \n,After Die-Attach,Work Instruction,Interconnect,Insufficient coverage could lead to:\ni. Weak ...
7,DAP05a,Void content,Void content (area of voids in %),Yes,X-RAY \n(not for non-conductive glue),min. Criteria:\nPackage criteria : Exposed pad...,10 dies per LF x 3 LFs \n,After Die-Attach,Work Instruction,Interconnect,Voids could lead to:\ni. Weakened die-bond for...
8,DAP05b,Void content and delamination (DAP05b is used ...,Void content (area of voids in %),Yes,SAM \n(preferred for leadfree solder wire and ...,min. Criteria:\nPackage criteria : Exposed pad...,10 dies per LF x 3 LFs \n,After Die-Attach,Work Instruction,Interconnect,Voids could lead to:\ni. Weakened die-bond for...
9,DAP05c,Solder joint crack / solder joint void,Appearance of crack,No,"Cross-Section (on lateral / vertical plane), F...",No crack,3 Units,After Molding,Lab Instruction\n,Nil,Voids and solder joint cracks could lead to:\n...


In [12]:
PBHB_df = pd.concat([PBHB_wirebond, PBHB_die_attach], ignore_index=True)
PBHB_df

,PB_code,requirements,parameter (input to pdr: output response),in adego rev. 7.0,verification method (input to pdr),development target (spec#) (input to pdr),sample size (input to pdr),take sample after process step,lab instruction (input in pdr),(primary) interaction to pb,motivation
0,INP01,Ball/bump bond dimension within bond pad opening,Ball/bump bond diameter\ni) Along X-axis\nii) ...,No,Light Optical Inspection,As per related specification requirement.\n\nT...,30 wires x number of lots @ each required proc...,After Interconnect,Work Instruction,Interconnect,"To avoid ball/bump bond out of bond pad, short..."
1,INP02,Wedge bond width within bond pad opening and l...,Wedge bond width\ni) At bond pad\nii) At lead ...,No,Light Optical Inspection,As per related specification requirement.\n(on...,30 wires x number of lots @ each required proc...,After Interconnect,Work Instruction,Interconnect,To avoid wedge bond out of bond pad/lead finge...
2,INP03,Stitch bond dimension within lead finger,Stitch bond dimension\ni) Stitch width\nii) St...,No,Light Optical Inspection,As per related specification requirement or MI...,30 wires x number of lots @ each required proc...,After Interconnect,Work Instruction,Interconnect,"To avoid broken stitch/wedge, lifted stitch/we..."
3,INP04,Verticle ball/bump bond dimension from bond pa...,Ball/bump bond height \n(Z-axis),Yes,Light Optical Inspection,As per related process specification requireme...,30 wires x number of lots @ each required proc...,After Interconnect,Work Instruction,Interconnect,"To avoid lifted ball/bump bond, smashed ball/b..."
4,INP05,Symmetrical/regular ball bond shape,Ball bond shape,Yes,Light Optical Inspection,Reject golf club formation if the center point...,30 wires x number of lots (shall include 4 di...,After Interconnect,Work Instruction,Interconnect,"To avoid lifted ball bond, smashed ball, dama..."
...,...,...,...,...,...,...,...,...,...,...,...
66,PDP005,C distribution enables good bonding,"C distribution (amount, thickness)",No,AES,C layer thickness < 1nm,1 Wafer: \n1 chip center\n1 chip edge,If Pad continues after passivation the samples...,Lab Instruction,Interconnect,"Bad bond adhesion (low ball shear, NSOP)"
67,PDP010,Native oxide thickness thin enough to ensure g...,"Thickness of native pad oxide (AlOx, CuOx)",No,AES,Alu pad with wedge bond/ Auball/ Cu ball bond:...,1 Wafer: \n1 chip center\n1 chip edge,If Pad continues after passivation the samples...,Lab Instruction,Interconnect,"Bad bond adhesion (low ball shear, NSOP), void..."
68,BSP08,Chip bow below critical limit,Chip Bow,No,Chip Bow Measurement,Warstein: <25µm: uncritical\nbetween 25 and 40...,"2 Wafers out of 2 lots:\n1 chip center, \n1 ch...",After Separation,Lab Instruction,Die-Attach,High chip bow can cause voids in the die attac...
69,SEP27,Die Strength requirements \n(according to deli...,Breakage force,No,Tool: Instron \n3-point bending method for det...,1) Industrie-Standard 3-point bend test and ba...,1 Wafer per Lot:\n100 dies w/ Frontside stress...,After Separation,Work Instruction,Thinning,No die cracks at customer/assembly or in field...


In [23]:
conn.execute("DROP TABLE PB_Handbook;")
conn.execute(table_to_sql(PBHB_df, 'PB_Handbook'))
PBHB_df.to_sql('PB_Handbook', conn, if_exists='replace', index=False)

In [13]:
PBHB_wirebond = pd.read_excel("../data/Z8F67581994_PBHB wire bond.xlsx", 
                              sheet_name="PB IN_Process", 
                              header=2,
                              usecols=range(1,8),
                              skiprows=[3]
                            )
PBHB_wirebond.rename(columns={'#': 'PB_code'}, inplace=True)
PBHB_wirebond['process_name'] = 'Interconnect'
PBHB_wirebond

,Au ball,Cu ball,Al wedge,Au wedge,Green & yellow,Red & grey,PB_code,process_name
0,X,X,NaN,NaN,X,NaN,INP01,Interconnect
1,NaN,NaN,X,X,X,NaN,INP02,Interconnect
2,X,X,NaN,NaN,X,NaN,INP03,Interconnect
3,X,X,NaN,NaN,X,NaN,INP04,Interconnect
4,X,X,NaN,NaN,X,NaN,INP05,Interconnect
5,X,X,NaN,NaN,X,NaN,INP06,Interconnect
6,X,X,NaN,NaN,X,NaN,INP07,Interconnect
7,NaN,NaN,X,X,X,NaN,INP08,Interconnect
8,X,X,NaN,NaN,X,NaN,INP09,Interconnect
9,X,X,NaN,NaN,X,NaN,INP10,Interconnect


In [14]:
PBHB_die_attach = pd.read_excel("../data/Z8F65720674_Die Attach.xlsx", 
                              sheet_name="PB DA_Process", 
                              header=2,
                              usecols=range(1,11),
                              skiprows=[3]
                            )
PBHB_die_attach.rename(columns={'Unnamed: 10': "PB_code"}, inplace=True)
PBHB_die_attach["PB_code"] = PBHB_die_attach["PB_code"].apply(lambda row: row.replace("\n", ""))
PBHB_die_attach['process_name'] = 'Die_Attach'
PBHB_die_attach

,Soft solder,Solder paste,Diffusion solder,Epoxy,DAF,Cu Pillar FC,Clip Bond,Green & yellow,Red & grey,PB_code,process_name
0,X,X,X,X,X,X,NaN,X,NaN,DAP01 /PUP01,Die_Attach
1,X,X,X,X,X,X,X,X,NaN,DAP02,Die_Attach
2,X,X,X,X,X,X,NaN,X,NaN,DAP03,Die_Attach
3,X,X,X,X,X,NaN,NaN,X,NaN,DAP04a,Die_Attach
4,NaN,NaN,NaN,NaN,NaN,X,NaN,X,NaN,DAP04b,Die_Attach
5,NaN,NaN,NaN,NaN,NaN,X,NaN,X,NaN,DAP04b,Die_Attach
6,NaN,NaN,NaN,NaN,NaN,NaN,X,X,NaN,DAP04c,Die_Attach
7,X,X,X,X,NaN,NaN,NaN,X,NaN,DAP05a,Die_Attach
8,X,X,X,X,X,NaN,NaN,X,NaN,DAP05b,Die_Attach
9,NaN,NaN,NaN,NaN,NaN,X,NaN,X,NaN,DAP05c,Die_Attach


In [15]:
PBHB_mapping = pd.concat([PBHB_wirebond, PBHB_die_attach], ignore_index=True)
for col in PBHB_mapping.columns:
    if col not in ['PB_code', 'process_name']:
        PBHB_mapping[col] = PBHB_mapping[col].apply(lambda row: row.replace("X", col) if not pd.isnull(row) else row)
PBHB_mapping = PBHB_mapping[['process_name', 'PB_code', 'Au ball', 'Cu ball', 'Al wedge', 'Au wedge', 
                             'Soft solder', 'Solder paste','Diffusion solder', 'Epoxy', 
                             'DAF', 'Cu Pillar FC', 'Clip Bond',
                             'Green & yellow', 'Red & grey']]
PBHB_mapping['PBC_color'] = PBHB_mapping[['Green & yellow', 'Red & grey']].apply(lambda x: x['Green & yellow'].lower().split(" & ") if pd.isnull(x['Red & grey']) \
                                                                                 else x['Red & grey'].lower().split(" & "), axis=1)
PBHB_mapping.drop(columns=['Green & yellow', 'Red & grey'], inplace=True)
PBHB_mapping = PBHB_mapping.explode(column="PBC_color", ignore_index=True)
PBHB_mapping.columns = ['process_name', "PB_code"] + ["process_tech_name_{i}".format(i=i) for i in range(len(PBHB_mapping.columns)-3)] + ["PBC_color"]
PBHB_mapping['process_tech_name'] = PBHB_mapping[[col for col in PBHB_mapping.columns if col.startswith("process_tech_name_")]].apply(lambda x: list(i for i in x if not pd.isnull(i)), axis=1)
PBHB_mapping.drop(columns=[col for col in PBHB_mapping.columns if col.startswith("process_tech_name_")], inplace=True)
PBHB_mapping = PBHB_mapping.explode(column="process_tech_name", ignore_index=True)
PBHB_mapping = PBHB_mapping.groupby(['process_name', 'process_tech_name', 'PBC_color'], as_index=False).agg({'PB_code': list})
# PBHB_mapping['PB_codes'] = PBHB_mapping.PB_code.apply(lambda x: ", ".join(x))
PBHB_mapping

,process_name,process_tech_name,PBC_color,PB_code
0,Die_Attach,Clip Bond,green,"[DAP02, DAP04c]"
1,Die_Attach,Clip Bond,grey,"[DAP09, DAP11 , DAP12, DAP14, PDP005, PDP010, ..."
2,Die_Attach,Clip Bond,red,"[DAP09, DAP11 , DAP12, DAP14, PDP005, PDP010, ..."
3,Die_Attach,Clip Bond,yellow,"[DAP02, DAP04c]"
4,Die_Attach,Cu Pillar FC,green,"[DAP01 /PUP01, DAP02, DAP03, DAP04b, DAP04b, D..."
5,Die_Attach,Cu Pillar FC,grey,"[DAP13, DAP15, BSP08, SEP27, SEP31]"
6,Die_Attach,Cu Pillar FC,red,"[DAP13, DAP15, BSP08, SEP27, SEP31]"
7,Die_Attach,Cu Pillar FC,yellow,"[DAP01 /PUP01, DAP02, DAP03, DAP04b, DAP04b, D..."
8,Die_Attach,DAF,green,"[DAP01 /PUP01, DAP02, DAP03, DAP04a, DAP05b, D..."
9,Die_Attach,DAF,grey,"[DAP05b, BSP08, SEP27, SEP31]"


In [18]:
process_table = pd.read_sql("select * from Process", conn)

In [19]:
Process_PBHB_mapping = process_table.merge(PBHB_mapping, on=['process_name', 'process_tech_name', 'PBC_color'], how='inner')[['process_id', 'PB_code']]
Process_PBHB_mapping.head(3)

,process_id,PB_code
0,2,"[DAP01 /PUP01, DAP02, DAP03, DAP04a, DAP05a, D..."
1,3,"[DAP01 /PUP01, DAP02, DAP03, DAP04a, DAP05a, D..."
2,4,"[DAP05b, DAP08, DAP10, DAP15, BSP08, SEP27, SE..."


In [20]:
Process_PBHB_mapping['PB_codes'] = Process_PBHB_mapping['PB_code'].apply(lambda row: [[i+1, x] for i,x in enumerate(row)])
Process_PBHB_mapping.drop(columns=['PB_code'], inplace=True)
Process_PBHB_mapping = Process_PBHB_mapping.explode(column='PB_codes', ignore_index=True)
Process_PBHB_mapping[['num','PB_code']] = pd.DataFrame(Process_PBHB_mapping.PB_codes.tolist(), index=Process_PBHB_mapping.index)

In [21]:
Process_PBHB_mapping.drop(columns=["PB_codes"], inplace=True)

In [22]:
Process_PBHB_mapping.head(3)

,process_id,num,PB_code
0,2,1,DAP01 /PUP01
1,2,2,DAP02
2,2,3,DAP03


In [23]:
# conn.execute("DROP TABLE Process_PBHB_mapping;")
conn.execute(table_to_sql(Process_PBHB_mapping, 'Process_PBHB_mapping', 
                          int_cols=['process_id', 'num'],
                         ))

Process_PBHB_mapping.to_sql('Process_PBHB_mapping', con=conn, 
                            if_exists='append', index=False)

conn.execute("ALTER TABLE Process_PBHB_mapping ADD FOREIGN KEY (process_id) REFERENCES Process(process_id);")

In [41]:
# conn.execute("drop table PDR_sessions;")
# conn.execute("drop table Sessions;")

query_sql = """
CREATE TABLE `Sessions` (
  `session_id` INT NOT NULL AUTO_INCREMENT,
  `submit_time` DATETIME,
  `username` varchar(50) DEFAULT NULL,
   PRIMARY KEY (session_id)
);
"""
conn.execute(query_sql)

query_sql = """
CREATE TABLE `PDR_sessions` (
  `session_id` int,
  `process_id` int,
  `PB_num` int DEFAULT NULL,
  `PB_code` varchar(18) DEFAULT NULL,
  `Process_results_Min` float DEFAULT NULL,
  `Process_results_Max` float DEFAULT NULL,
  `Process_results_Ave` float DEFAULT NULL,
  `Process_results_sigma` float DEFAULT NULL,
  `Process_results_attribute` varchar(50) DEFAULT NULL,
  `Target_achieved` boolean DEFAULT NULL,
  `Rec_action_number` varchar(20) DEFAULT NULL,
  FOREIGN KEY (session_id) REFERENCES Sessions(session_id),
  FOREIGN KEY (process_id) REFERENCES Process(process_id)
);
"""
conn.execute(query_sql)

# GPPT

In [24]:
GPPT_templates_wirebond = pd.read_excel("../data/wire bond GPPT.xlsx", 
                                        sheet_name="Template Overview",
                                       )
GPPT_templates_wirebond.columns = ['Template', 'Machine_Model', 'Remark']
GPPT_templates_wirebond['Machine_Model'] = GPPT_templates_wirebond['Machine_Model'].str.split("\n ")
GPPT_templates_wirebond = GPPT_templates_wirebond.explode(column="Machine_Model", ignore_index=True)
GPPT_templates_wirebond

,Template,Machine_Model,Remark
0,ASM Template #1,iHawk Xtreme,Ball Bonder
1,ASM Template #1,iHawk Xtreme GoCu,Ball Bonder
2,ASM Template #1,iHawk Xpress GoCu,Ball Bonder
3,ASM Template #1,Eagle Xtreme GoCu,Ball Bonder
4,ASM Template #1,Eagle AERO,Ball Bonder
5,ASM Template #1,iHawk,Ball Bonder
6,ASM Template #2,Eagle 60,Ball Bonder
7,Shinkawa Template #1,ACB-1000,Ball Bonder
8,Shinkawa Template #1,ACB-3000,Ball Bonder
9,Shinkawa Template #1,UTC-1000,Ball Bonder


In [25]:
GPPT_templates_dieattach = pd.read_excel("../data/Die attach GPPT.xlsx", 
                                         sheet_name="FrontPage",
                                         skiprows=list(range(4))+list(range(20,25)),
                                         usecols=[1,2]
                                       )
GPPT_templates_dieattach['Remark'] = None
GPPT_templates_dieattach.columns = ['Machine_Model', 'Template', 'Remark']
GPPT_templates_dieattach['Machine_Model'] = GPPT_templates_dieattach['Machine_Model'].str.split(", ")
GPPT_templates_dieattach = GPPT_templates_dieattach.explode(column="Machine_Model", ignore_index=True)
GPPT_templates_dieattach

,Machine_Model,Template,Remark
0,AD8312,ASM-AD8312/FC,None
1,AD8312FC,ASM-AD8312/FC,None
2,AD838,ASM-AD838,None
3,Texus 4000,TEXUS-4600/4200/4000,None
4,Texus 4200,TEXUS-4600/4200/4000,None
5,Texus 4600,TEXUS-4600/4200/4000,None
6,ESEC2100,BESI-ESEC2100,None
7,BESI Datacon Evo,BESI-Datacon Evo,None
8,ESEC 2007,BESI-ESEC2007/2009,None
9,ESEC 2009,BESI-ESEC2007/2009,None


In [26]:
GPPT_machine_templates_table = pd.concat([GPPT_templates_wirebond, GPPT_templates_dieattach], ignore_index=True)
GPPT_machine_templates_table

,Template,Machine_Model,Remark
0,ASM Template #1,iHawk Xtreme,Ball Bonder
1,ASM Template #1,iHawk Xtreme GoCu,Ball Bonder
2,ASM Template #1,iHawk Xpress GoCu,Ball Bonder
3,ASM Template #1,Eagle Xtreme GoCu,Ball Bonder
4,ASM Template #1,Eagle AERO,Ball Bonder
5,ASM Template #1,iHawk,Ball Bonder
6,ASM Template #2,Eagle 60,Ball Bonder
7,Shinkawa Template #1,ACB-1000,Ball Bonder
8,Shinkawa Template #1,ACB-3000,Ball Bonder
9,Shinkawa Template #1,UTC-1000,Ball Bonder


In [28]:
# conn.execute("DROP TABLE GPPT_machine_template;")
conn.execute(table_to_sql(GPPT_machine_templates_table, 'GPPT_machine_template'))

GPPT_machine_templates_table.to_sql('GPPT_machine_template', con=conn, 
                            if_exists='append', index=False)

In [29]:
wirebond_GPPT_templates = list(GPPT_templates_wirebond.Template.unique())
GPPT_wirebond_template = []
for sheet in wirebond_GPPT_templates:
    try:
        df_temp = pd.read_excel("../data/wire bond GPPT.xlsx", 
                            sheet_name=sheet,
                            usecols=[2,3,4],
                            skiprows=range(pd.read_excel("../data/wire bond GPPT.xlsx", 
                                              sheet_name=sheet,
                                              usecols=[1]
                                             )['Unnamed: 1'].tolist().index('Main Parameter')+2),
                           )

        df_temp.dropna(subset=['Parameter Name'], inplace=True)
        df_temp.columns = ['Work Step', 'Num', 'Parameter Name']
        df_temp.fillna(method='ffill', inplace=True)
        df_temp['Template'] = sheet
        GPPT_wirebond_template.append(df_temp)
    except ValueError: print(sheet)
GPPT_wirebond_template = pd.concat(GPPT_wirebond_template, ignore_index=True)[["Template", 'Work Step', 'Num', 'Parameter Name']]
GPPT_wirebond_template.head(20)

K&S Template #3
Hesse Template #2


,Template,Work Step,Num,Parameter Name
0,ASM Template #1,Heating,1.1,Pre Heater
1,ASM Template #1,Heating,1.2,Main Heater
2,ASM Template #1,Heating,1.3,Post Heater
3,ASM Template #1,Indexer Gas Flow \n(if applicable),2.1,Buffer
4,ASM Template #1,Indexer Gas Flow \n(if applicable),2.2,PH & Cvr
5,ASM Template #1,Indexer Gas Flow \n(if applicable),2.3,H/Tunnel
6,ASM Template #1,Indexer Gas Flow \n(if applicable),2.4,Heater
7,ASM Template #1,Indexer Gas Flow \n(if applicable),2.5,W/Clamp
8,ASM Template #1,Indexer Gas Flow \n(if applicable),2.6,PB & Cvr
9,ASM Template #1,Cu Kit Gas Flow \n(if applicable),3.1,E Torch


In [30]:
dieattach_GPPT_templates = [sheet for sheet in pd.ExcelFile('../data/Die attach GPPT.xlsx').sheet_names if sheet != 'FrontPage']
GPPT_dieattach_template = []
for sheet in dieattach_GPPT_templates:
    df_temp = pd.read_excel("../data/Die attach GPPT.xlsx", 
                                sheet_name=sheet,
                                usecols=[2,3,4],
                                skiprows=range(pd.read_excel("../data/Die attach GPPT.xlsx", 
                                              sheet_name=sheet,
                                              usecols=[1],
                                             )['Unnamed: 1'].tolist().index('Main Parameter')+2),
                               )
    try:
        df_temp.dropna(subset=['Parameter Name'], inplace=True)
        df_temp.columns = ['Work Step', 'Num', 'Parameter Name']
        df_temp.fillna(method='ffill', inplace=True)
        df_temp['Template'] = sheet.split(maxsplit=1)[-1].strip()
        GPPT_dieattach_template.append(df_temp)
    except ValueError: print(sheet)
GPPT_dieattach_template = pd.concat(GPPT_dieattach_template, ignore_index=True)[["Template", 'Work Step', 'Num', 'Parameter Name']]
GPPT_dieattach_template.head(20)

,Template,Work Step,Num,Parameter Name
0,ASM-AD8312_FC,Pick Up Head ( Flipper ),3.1,Pick delay (ms)
1,ASM-AD8312_FC,Pick Up Head ( Flipper ),3.2,Expansion Setting
2,ASM-AD8312_FC,Pick Up Head ( Flipper ),3.3,Ejector up level (um)
3,ASM-AD8312_FC,Pick Up Head ( Flipper ),3.4,Ejector up delay (ms)
4,ASM-AD8312_FC,Pick Up Head ( Flipper ),3.5,Sync Pick speed (ms)
5,ASM-AD8312_FC,Pick Up Head ( Flipper ),3.6,Pick search heigh (um)
6,ASM-AD8312_FC,Pick Up Head ( Flipper ),3.7,Pick search delay (ms)
7,ASM-AD8312_FC,Pick Up Head ( Flipper ),4.1,Pick delay (ms)
8,ASM-AD8312_FC,Pick Up Head ( Flipper ),4.2,Pick force
9,ASM-AD8312_FC,Bond Head,4.3,Sync Pick speed (ms)


In [31]:
len(GPPT_wirebond_template),len(GPPT_dieattach_template)

(692, 367)

In [32]:
GPPT_template_parameters_table = pd.concat([GPPT_wirebond_template, GPPT_dieattach_template], ignore_index=True)
GPPT_template_parameters_table

,Template,Work Step,Num,Parameter Name
0,ASM Template #1,Heating,1.1,Pre Heater
1,ASM Template #1,Heating,1.2,Main Heater
2,ASM Template #1,Heating,1.3,Post Heater
3,ASM Template #1,Indexer Gas Flow \n(if applicable),2.1,Buffer
4,ASM Template #1,Indexer Gas Flow \n(if applicable),2.2,PH & Cvr
...,...,...,...,...
1054,ASM-AD8312+,Bonding,5.1,Bond Delay
1055,ASM-AD8312+,Bonding,5.2,Bond Force
1056,ASM-AD8312+,Bonding,5.3,Bond level Offset
1057,ASM-AD8312+,Bonding,5.4,Bond Search Level


In [33]:
# conn.execute("DROP TABLE GPPT_template_parameters;")
conn.execute(table_to_sql(GPPT_template_parameters_table, 'GPPT_template_parameters', float_cols=["Num"]))

GPPT_template_parameters_table.to_sql('GPPT_template_parameters', con=conn, 
                            if_exists='append', index=False)

#### resources
- https://stackoverflow.com/questions/68852940/how-to-create-two-dependent-dynamic-dropdown-lists-using-flask
- https://www.geeksforgeeks.org/connect-flask-to-a-database-with-flask-sqlalchemy/?ref=rp
- https://stackoverflow.com/questions/63434179/how-to-read-a-custom-html-table-information-from-client-side-into-flask-backend
